In [1]:
print('hello world')

hello world


In [2]:
import numpy as np

a_data = np.arange(10)
print('array', a_data)

deleted_index = np.where(a_data == 5)[0][0]

a_data = np.delete(a_data, deleted_index)
print(deleted_index)
print(a_data)

array [0 1 2 3 4 5 6 7 8 9]
5
[0 1 2 3 4 6 7 8 9]


In [3]:
# Count number of instance in an array
arr = np.array([6, 5, 7, 1, 9, 7, 7, 2])

seven_count = (arr == 7).sum()
print(seven_count)

3


In [4]:
# How much faster is numpy array comparing to python list?
import numpy as np
import time


def multiply_lists(size=100_000):
    # Generate 2 lists
    list1 = list(range(size))
    list2 = list1.copy()

    return [(a * b) for a, b in zip(list1, list2)]

def multiply_arrays(size=100_000):
    arr1 = np.arange(size)
    arr2 = arr1.copy()

    return arr1 * arr2

def compare_performance(size=100_000):
    time_elapsed_list = 0
    time_elapsed_array = 0

    # Capture start time for list operations
    start_time = time.time()

    # Run multiplication for lists
    multiply_lists(size)

    # Calculate elapsed time
    time_elapsed_list = time.time() - start_time

    # Capture stat time for array operations
    start_time = time.time()

    # Run multiplication for arrays
    multiply_arrays(size)
    
    # Calculate elapsed time
    time_elapsed_array = time.time() - start_time

    # How much faster
    faster_rate = time_elapsed_list / time_elapsed_array

    # Print out results
    print(f'Multiplication 2 lists of size {size:,}: {time_elapsed_list} ms.')
    print(f'Multiplication 2 arrays of size {size:,}: {time_elapsed_array} ms.')
    print(f"Numpy array operation is faster {faster_rate:.2f} times")


compare_performance(10000000)

Multiplication 2 lists of size 10,000,000: 1.4519853591918945 ms.
Multiplication 2 arrays of size 10,000,000: 0.04001617431640625 ms.
Numpy array operation is faster 36.28 times


In [5]:
# Implement a function to compute Intersection over union - iou

def compute_iou(box_a, box_b):
    # Calculate areas of 2 boxes
    area_box_a = (box_a[2] - box_a[0] + 1) * (box_a[3] - box_a[1] + 1)
    area_box_b = (box_b[2] - box_b[0] + 1) * (box_b[3] - box_b[1] + 1)

    # Locate coordinates of the intersection
    x_a = max(box_a[0], box_b[0])
    y_a = max(box_a[1], box_b[1])
    x_b = min(box_a[2], box_b[2])
    y_b = min(box_a[3], box_b[3])

    # Compute intersection area
    area_intersection = max(0, (x_b - x_a + 1)) * max(0, (y_b - y_a + 1))

    # Comput iou
    area_union = area_box_a + area_box_b - area_intersection
    iou = area_intersection / area_union

    return iou

# Tests
box_a = [0, 0, 100, 100]
box_b = [50, 50, 150, 150]

print(compute_iou(box_a, box_b))

0.14611538677602381


In [10]:
# Compute iou for N-bounding boxes - using list

def time_taken_lists(size=10_000):
    boxes = []

    for i in range(size):
        box = [i+1] * 4
        boxes.append(box)
    
    # Compute iou for size - 1 bounding boxes
    ground_truth_box = boxes[0]

    # Capture start time
    start_time = time.time()

    for i in range(1, len(boxes)):
        iou = compute_iou(ground_truth_box, boxes[i])

    # Time taken
    elapsed_time = time.time() - start_time

    print('Elapsed time:', elapsed_time, " miliseconds")

time_taken_lists()



Elapsed time: 0.025608539581298828  miliseconds


In [12]:
import time
import numpy as np


def time_taken_array(size=10_000):
    # Capture start time
    start_time = time.time()

    # Generate boxes
    boxes = [[i]*4 for i in range(size)]
    box_array = np.array(boxes)
    
    # Coordinates of bounding boxes (999)
    x1 = box_array[:, 0]
    y1 = box_array[:, 1]
    x2 = box_array[:, 2]
    y2 = box_array[:, 3]

    # Compute areas of bounding boxes
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)

    # Compute intersection areas
    xx1 = np.maximum(x1[0], x1[1:])
    yy1 = np.maximum(y1[0], y1[1:])
    xx2 = np.minimum(x2[0], x2[1:])
    yy2 = np.minimum(y2[0], y2[1:])

    w = np.maximum(0, xx2 - xx1 + 1)
    h = np.maximum(0, yy2 - yy1 + 1)

    inter_area = w * h

    # Compute iou
    ious = inter_area / (areas[0] + areas[1:] - inter_area)

    # Compute time taken
    time_taken = time.time() - start_time
    print(f"Time taken: {time_taken} milliseconds")




print('Processing arrays...')
time_taken_array()
print('Processing lists...')
time_taken_lists()

Processing arrays...
Time taken: 0.012019157409667969 milliseconds
Processing lists...
Elapsed time: 0.019542932510375977  miliseconds


In [24]:
# Choose which bounding boxes to eliminate

def non_max_suppression(boxes, scores, iou_threshold):

    # Sort the boxes by score in ascending order
    sorted_indices = sorted(range(len(boxes)), key=lambda x: scores[x], reverse=True)

    # List ot stroe the indes of boxes to keep
    keep = []

    while sorted_indices:
        i = sorted_indices.pop(0) # index of the current highest score box
        keep.append(i)

        filtered_indices = []
        for j in sorted_indices:
            iou = compute_iou(boxes[i], boxes[j])
            if iou <= iou_threshold:
                filtered_indices.append(j)
        sorted_indices = filtered_indices

    return keep


# Tests

boxes = [
    [12, 84, 140, 212],
    [24, 84, 152, 212],
    [36, 84, 164, 212],
    [12, 96, 140, 224],
    [24, 96, 152, 224]
]

scores = [0.3, 0.4, 0.5, 0.6, 0.7]
iou_threshold = 0.3
print(non_max_suppression(boxes, scores, iou_threshold))


[4]


In [35]:
# Implement non_max_suppression using numpy arrays

def non_max_suppression_arr(boxes, scores, iou_threshold):

    if len(boxes) == 0:
        return []
    
    # Coordinates of bounding boxes
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    # Compute area of bounding boxes
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)

    # Sort by scores
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)

        # Compute iou
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        intersection = w * h
        iou = intersection / (areas[i] + areas[order[1:]] - intersection)

        # Suppress bounding boexes with iou over the threshold
        inds = np.where(iou <= iou_threshold)[0]
        order = order[inds + 1]
    return keep
boxes_array = np.array(boxes)
scores_array = np.array(scores)
non_max_suppression_arr(boxes_array, scores_array, iou_threshold)


[4]